<a href="https://colab.research.google.com/github/DCajiao/Aplicaciones-del-Analisis-Espacial/blob/main/casos/Situaci%C3%B3n_2/00_preprocesamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import requests
import pandas as pd
import numpy as np
from io import BytesIO

library_url = "https://raw.githubusercontent.com/DCajiao/workshop003_Machine_learning_and_Data_streaming/refs/heads/main/src/utils/analysis_functions.py"
analysis_functions_py = requests.get(library_url).text
with open("analysis_functions.py", "w") as f:
    f.write(analysis_functions_py)

import analysis_functions as af


In [51]:
# Importación de Datos
DATA_GITHUB_URL = f'https://raw.githubusercontent.com/DCajiao/Aplicaciones-del-Analisis-Espacial/refs/heads/main/casos/Situación_2/data/AccidentesFatales'#_{año}.xlsx'

df_2009 = pd.read_excel(BytesIO(requests.get(f'{DATA_GITHUB_URL}_2009.xlsx').content), engine='openpyxl')
df_2010 = pd.read_excel(BytesIO(requests.get(f'{DATA_GITHUB_URL}_2010.xlsx').content), engine='openpyxl')

In [52]:
# print(df_2009.columns)
# print(df_2010.columns)

# Tienen las mismas columnas?
print(set(df_2009.columns) - set(df_2010.columns))
print(set(df_2010.columns) - set(df_2009.columns))

set()
{'FECHA INSP.'}


In [53]:
# Eliminar la columna FECHA INSP.
df_2010.drop(columns=['FECHA INSP.'], inplace=True)

In [54]:
# Hacer un merge de los datos creando la etiqueta AÑO_DATA:
df_2009['AÑO_DATA'] = 2009
df_2010['AÑO_DATA'] = 2010

df = pd.concat([df_2009, df_2010])

In [55]:
af.summary_by_columns(df)

,Column,Data Type,Missing Values,Unique Values,Duplicates,Missing Values (%)
0,MES FALLECIMIENTO,object,0,12,590,0.00
1,MES ACCIDENTE,object,0,13,589,0.00
2,BARRIO,object,0,263,339,0.00
3,COM,int64,0,22,580,0.00
4,PROFESION,object,66,138,463,10.96
5,SEXO,object,2,2,599,0.33
6,EDAD,float64,10,84,517,1.66
7,EDAD AGRUPADA,object,10,18,583,1.66
8,FECHA ACCIDENTE,datetime64[ns],0,407,195,0.00
9,FECHA FALLECIMIENTO,datetime64[ns],0,393,209,0.00


In [56]:
df.size

13846

In [57]:
# Eliminar registros con valores nulos en SEXO, EDAD AGRUPADA
df.dropna(subset=['SEXO', 'EDAD AGRUPADA'], inplace=True)

In [58]:
af.summary_by_columns(df)

,Column,Data Type,Missing Values,Unique Values,Duplicates,Missing Values (%)
0,MES FALLECIMIENTO,object,0,12,580,0.00
1,MES ACCIDENTE,object,0,13,579,0.00
2,BARRIO,object,0,261,331,0.00
3,COM,int64,0,22,570,0.00
4,PROFESION,object,57,138,453,9.63
5,SEXO,object,0,2,590,0.00
6,EDAD,float64,0,84,508,0.00
7,EDAD AGRUPADA,object,0,18,574,0.00
8,FECHA ACCIDENTE,datetime64[ns],0,401,191,0.00
9,FECHA FALLECIMIENTO,datetime64[ns],0,389,203,0.00


In [59]:
df.columns

Index(['MES FALLECIMIENTO', 'MES ACCIDENTE', 'BARRIO', 'COM', 'PROFESION',
       'SEXO', 'EDAD', 'EDAD AGRUPADA', 'FECHA ACCIDENTE',
       'FECHA FALLECIMIENTO', 'HORA FALLECIMIENTO', 'HORA ACCIDENTE',
       'DIA SEMANA FALLECIMIENTO', 'DIA SEMANA ACCIDENTE', 'LUGAR INSP.',
       'CONDICCION', 'VEHICULOS', 'Ciudad', 'coordenada X (metros)',
       'coordenada Y (metros)', 'coordenada X (km)', 'coordenada Y (km)',
       'AÑO_DATA'],
      dtype='object')

In [60]:
# ¿Cuantos vehículos únicos hay en el dataset?
print(df['CONDICCION'].nunique())


25


In [61]:
# Diccionario de estandarización
map_cond = {
    # Conductores
    "COND. MOTO": "CONDUCTOR MOTO",
    "COND. MOTOCARRO": "CONDUCTOR MOTOCARRO",
    "COND. AUTO": "CONDUCTOR AUTOMOVIL",
    "COND. TAXI": "CONDUCTOR TAXI",
    "COND.TAXI": "CONDUCTOR TAXI",
    "CONDUCTOR TAXI": "CONDUCTOR TAXI",
    "CONDUCTOR VOLQUETA": "CONDUCTOR VOLQUETA",
    "CONDUCTOR": "CONDUCTOR AUTOMOVIL",
    
    # Pasajeros
    "PASAJERO": "PASAJERO",
    "PASAJERA": "PASAJERO",
    "PASAJERO AUTO": "PASAJERO AUTOMOVIL",
    "PAS. AUTOMOVIL": "PASAJERO AUTOMOVIL",
    "PAS. AUTO": "PASAJERO AUTOMOVIL",
    "PAS. MOTO": "PASAJERO MOTO",
    "PAS. VOLQUETA": "PASAJERO VOLQUETA",
    "PAS. BUSETA": "PASAJERO BUSETA",
    "PAS. CAMPERO": "PASAJERO CAMPERO",
    "PAS. MICROBUS": "PASAJERO MICROBUS",
    "PAS. BUS": "PASAJERO BUS",
    "PAS. TAXI": "PASAJERO TAXI",
    "PAS. CAMION ": "PASAJERO CAMION",
    "PAS. BICICLETA": "PASAJERO BICICLETA",
    
    # Otros
    "PEATON": "PEATON",
    "PEATON ": "PEATON",
    "CICLISTA": "CICLISTA"
}

# Normalizar la columna
df["CONDICCION_STD"] = df["CONDICCION"].replace(map_cond)

# Ver categorías únicas estandarizadas
print("Categorías únicas estandarizadas:")
print(df["CONDICCION_STD"].unique())

# Contar vehículos únicos
n_unicos = df["CONDICCION_STD"].nunique()
print(f"\nNúmero de categorías únicas estandarizadas: {n_unicos}")


Categorías únicas estandarizadas:
['CONDUCTOR MOTO' 'CICLISTA' 'PEATON' 'PASAJERO' 'PASAJERO MOTO'
 'CONDUCTOR AUTOMOVIL' 'PASAJERO AUTOMOVIL' 'PASAJERO VOLQUETA'
 'PASAJERO BUSETA' 'PASAJERO CAMPERO' 'CONDUCTOR MOTOCARRO'
 'PASAJERO MICROBUS' 'CONDUCTOR TAXI' 'CONDUCTOR VOLQUETA'
 'PASAJERO BICICLETA' 'PASAJERO BUS' 'PASAJERO TAXI' 'PASAJERO CAMION']

Número de categorías únicas estandarizadas: 18


In [62]:
# Guardado de Datos
df.to_csv('data/Accidentes_2009_2010.csv', index=False)